# Оцениваем регрессии на данных HS0 в спецификациях:
- Сырые данные ~ ковид и ограничения + контроль на дамми
- Сырые данные ~ ковид и ограничения + контроль на дамми | инструментальные переменные

Для второй спецификации считаем бутстраповские доверительные интервалы

In [12]:
library(arrow)
library(stargazer)
library(tidyverse)
library(plotly)
library(fixest)
signif.code = signif.code = c("***" = 0.001, "**" = 0.01, "*" = 0.05, "." = 0.10)

## Сырые данные

Под каждой регрессией вывожу summary с кластеризацией ошибок. В конце блока сравниваю робастность результата по используемым ошибкам.

In [4]:
# Скачиваю данные и переназываю переменные
df = read_parquet('../data/processed_data/trade_hs0_x_covid.parquet.gzip')
# Переменная pair нужна, чтобы считать НАС стандартные ошибки
df = df %>% mutate(pair = paste(i, j))

df$deaths_i = df$deaths
df$deaths_j = df$`deaths:r_j`

df$median_age_i = df$median_age
df$median_age_j = df$`median_age:r_j`

df$average_surface_temperature_i = df$average_surface_temperature
df$average_surface_temperature_j = df$`average_surface_temperature:r_j`

df$C1M_School_closing_i = df$C1M_School_closing
df$C1M_School_closing_j = df$`C1M_School_closing:r_j`

df$population_density_i = df$population_density
df$population_density_j = df$`population_density:r_j`

df$StringencyIndex_Average_i = df$StringencyIndex_Average
df$StringencyIndex_Average_j = df$`StringencyIndex_Average:r_j`

df$share_1_members_i = df$share_1_members
df$share_1_members_j = df$`share_1_members:r_j`

df$H6M_Facial_Coverings_i = df$H6M_Facial_Coverings
df$H6M_Facial_Coverings_j = df$`H6M_Facial_Coverings:r_j`

df$EconomicSupportIndex_i = df$EconomicSupportIndex
df$EconomicSupportIndex_j = df$`EconomicSupportIndex:r_j`

df$C8EV_International_travel_controls_i = df$C8EV_International_travel_controls
df$C8EV_International_travel_controls_j = df$`C8EV_International_travel_controls:r_j`
df %>% head()

t,i,j,v,q,v19,q19,country,t:r_i,cases,⋯,StringencyIndex_Average_i,StringencyIndex_Average_j,share_1_members_i,share_1_members_j,H6M_Facial_Coverings_i,H6M_Facial_Coverings_j,EconomicSupportIndex_i,EconomicSupportIndex_j,C8EV_International_travel_controls_i,C8EV_International_travel_controls_j
<int>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<int>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
2021,LUX,LBY,968.018,210.173,501.858,127.012,LUX,2021,241.1512329,⋯,46.04334,63.24680,33.34,NA,2.0082192,2.827397,43.184933,0.00000,2.534247,2.520548
2021,RUS,SEN,1307165.125,3131615.500,789625.305,2232100.632,RUS,2021,138.1677534,⋯,46.46488,24.90279,25.69,6.92,2.3150685,3.402740,50.753426,0.00000,2.610959,2.561644
2020,SOM,KOR,3136.611,2045.876,5608.268,4002.770,SOM,2020,0.7276796,⋯,34.65910,49.42956,1.53,31.75,0.7622951,1.653005,2.254098,38.72951,2.101093,2.415301
2020,TUR,CRI,57792.285,98850.453,60858.013,86163.873,TUR,2020,67.6935912,⋯,54.57148,54.39423,6.37,12.07,2.2732241,1.218579,60.211750,37.84153,2.040984,2.322404
2021,KNA,BIH,0.520,0.008,0.460,0.005,KNA,2021,164.5090411,⋯,NA,38.78658,NA,11.50,NA,2.476712,NA,44.96575,NA,1.000000
2022,AZE,MLT,22060.938,39007.156,12476.490,33493.400,AZE,2022,56.3560822,⋯,49.01625,22.13003,6.77,22.64,0.6575342,2.323288,9.863013,15.51370,3.000000,1.263014


In [13]:
# FE (т.е. оценка на панельных данных с использованием внутригрупповой дисперсии)
fe_no_iv = feols(log(v/v19) ~ log(deaths_i) + log(deaths_j) +
#  C8EV_International_travel_controls_i + C8EV_International_travel_controls_j + 
 StringencyIndex_Average_i + StringencyIndex_Average_j+ 
 EconomicSupportIndex_i + EconomicSupportIndex_j| i + j + year,
 data = df)
fe_no_iv %>% summary(cluster = ~ i+j)

NOTE: 13,568 observations removed because of NA and infinite values (RHS: 13,568).



OLS estimation, Dep. Var.: log(v/v19)
Observations: 56,739
Fixed-effects: i: 175,  j: 175,  year: 3
Standard-errors: Clustered (i & j) 
                           Estimate Std. Error   t value Pr(>|t|)    
log(deaths_i)             -0.011664   0.005799 -2.011600 0.045805 *  
log(deaths_j)              0.001111   0.005060  0.219590 0.826448    
StringencyIndex_Average_i -0.001249   0.000833 -1.500382 0.135328    
StringencyIndex_Average_j -0.001689   0.000789 -2.140834 0.033680 *  
EconomicSupportIndex_i    -0.000649   0.000405 -1.604036 0.110520    
EconomicSupportIndex_j    -0.000057   0.000354 -0.161464 0.871915    
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1
RMSE: 0.981472     Adj. R2: 0.038429
                 Within R2: 4.537e-4

In [14]:
# Добавляю гетерогенность эффектов во времени
heterogenous_fe = feols(log(v/v19) ~ year/log(deaths_i) + year/log(deaths_j) +
# year/C8EV_International_travel_controls_i + year/C8EV_International_travel_controls_j + 
year/StringencyIndex_Average_i + year/StringencyIndex_Average_j + 
year/EconomicSupportIndex_i + year/EconomicSupportIndex_j|
i + j ,
data = df)
heterogenous_fe %>% summary(cluster = ~ i+j)

NOTE: 13,568 observations removed because of NA and infinite values (RHS: 13,568).



OLS estimation, Dep. Var.: log(v/v19)
Observations: 56,739
Fixed-effects: i: 175,  j: 175
Standard-errors: Clustered (i & j) 
                                    Estimate Std. Error   t value   Pr(>|t|)
year2021                           -0.078750   0.087680 -0.898147 0.37034849
year2022                           -0.138452   0.117884 -1.174482 0.24180688
year2020:log(deaths_i)             -0.003142   0.006363 -0.493806 0.62206614
year2021:log(deaths_i)             -0.000548   0.006579 -0.083230 0.93376468
year2022:log(deaths_i)             -0.016525   0.007314 -2.259296 0.02510570
year2020:log(deaths_j)             -0.002142   0.006636 -0.322756 0.74726791
year2021:log(deaths_j)             -0.000596   0.006193 -0.096176 0.92349137
year2022:log(deaths_j)              0.003218   0.007229  0.445212 0.65671986
year2020:StringencyIndex_Average_i -0.004521   0.001335 -3.386177 0.00087616
year2021:StringencyIndex_Average_i -0.002291   0.001021 -2.243425 0.02613087
year2022:StringencyIndex_Av

### Инструментальные переменные на панельных данных

In [15]:
# Спецификация с фиксированными эффектами, куда я добавил инструменты, чтобы убрать потенциальную двустороннюю связь. 

fe_iv = feols(log(v/v19) ~  1 +
#  C8EV_International_travel_controls_i + C8EV_International_travel_controls_j +
 StringencyIndex_Average_i + StringencyIndex_Average_j+
 EconomicSupportIndex_i + EconomicSupportIndex_j
 | year+i+j|
 log(deaths_i) + log(deaths_j) ~ 
# H6M_Facial_Coverings_i+H6M_Facial_Coverings_j
# +
C1M_School_closing_i+C1M_School_closing_j
+
median_age_i + median_age_j
+
population_density_i + population_density_j
# +
# average_surface_temperature_i + average_surface_temperature_j # Добавляет пропусков

 ,
 data = df
 )
fe_iv %>% summary(cluster = ~ i+j)

NOTE: 13,906 observations removed because of NA and infinite values (RHS: 9,802, IV: 6,067/10,661).



TSLS estimation - Dep. Var.: log(v/v19)
                  Endo.    : log(deaths_i), log(deaths_j)
                  Instr.   : C1M_School_closing_i, C1M_School_closing_j, median_age_i, median_age_j, population_density_i, population_density_j
Second stage: Dep. Var.: log(v/v19)
Observations: 56,401
Fixed-effects: year: 3,  i: 175,  j: 175
Standard-errors: Clustered (i & j) 
                           Estimate Std. Error   t value Pr(>|t|)    
fit_log(deaths_i)         -0.065089   0.033134 -1.964411 0.051075 .  
fit_log(deaths_j)          0.055612   0.038818  1.432637 0.153756    
StringencyIndex_Average_i -0.000047   0.001174 -0.040263 0.967929    
StringencyIndex_Average_j -0.002824   0.001325 -2.132414 0.034375 *  
EconomicSupportIndex_i    -0.000745   0.000459 -1.621591 0.106702    
EconomicSupportIndex_j     0.000072   0.000441  0.164377 0.869625    
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1
RMSE: 0.983068     Adj. R2:  0.034393
                 Within R2: -

### Экспорт результатов в латех

In [16]:
etable(fe_no_iv, heterogenous_fe, fe_iv, vcov = ~i+j, signif.code=signif.code, fitstat=~ . + sargan.p + wh.p + ivf, tex=FALSE)

,,fe_no_iv,heterogenous_fe,fe_iv
,<chr>,<chr>,<chr>,<chr>
1,Dependent Var.:,log(v/v19),log(v/v19),log(v/v19)
2,,,,
3,log(deaths_i),-0.0117* (0.0058),,-0.0651. (0.0331)
4,log(deaths_j),0.0011 (0.0051),,0.0556 (0.0388)
5,StringencyIndex_Average_i,-0.0012 (0.0008),,-4.73e-5 (0.0012)
6,StringencyIndex_Average_j,-0.0017* (0.0008),,-0.0028* (0.0013)
7,EconomicSupportIndex_i,-0.0006 (0.0004),,-0.0007 (0.0005)
8,EconomicSupportIndex_j,-5.71e-5 (0.0004),,7.24e-5 (0.0004)
9,year2021,,-0.0788 (0.0877),


In [17]:
etable(fe_no_iv, heterogenous_fe, fe_iv, vcov = ~i+j, signif.code=signif.code, fitstat=~ . + sargan.p + wh.p + ivf, tex=TRUE)

\begingroup
\centering
\begin{tabular}{lccc}
   \tabularnewline \midrule \midrule
   Dependent Variable: & \multicolumn{3}{c}{log(v/v19)}\\
   Model:                                           & (1)                    & (2)                   & (3)\\  
   \midrule
   \emph{Variables}\\
   log(deaths\_i)                                   & -0.0117$^{*}$          &                       & -0.0651$^{.}$\\   
                                                    & (0.0058)               &                       & (0.0331)\\   
   log(deaths\_j)                                   & 0.0011                 &                       & 0.0556\\   
                                                    & (0.0051)               &                       & (0.0388)\\   
   StringencyIndex\_Average\_i                      & -0.0012                &                       & $-4.73\times 10^{-5}$\\    
                                                    & (0.0008)               &                       & (0.0012)\\